In [ ]:
!pip install geemap
import ee, geemap
ee.Authenticate()
ee.Initialize()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=dBXUDRtqOmRBmKr2f6tZAUkdsXd0Lkq3-SfvzlLIwSY&tc=Nu68PIMshbJNhK7GsZCYIJQcuflw_zJvaQHmL1Xi-_c&cc=diHCLZvbeiGDSa163HwX23EOH5_8yzFbaCPhltMvvOg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh73eaGhBN6blUK9Dh5JJKHfkuf5st75G0sywhgwxQGcoQ1SzR3vneo

Successfully saved authorization token.


In [37]:
import folium


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [38]:
image = ee.Image('LANDSAT/LT05/C01/T1_TOA/LT05_015030_20100531')

print(image.projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:32618', 'transform': [30, 0, 337485, 0, -30, 4888515]}


In [39]:
print('Pixel Size in meters: ', image.projection().nominalScale().getInfo())

Pixel Size in meters:  30


In [40]:
image = ee.Image('MODIS/MOD13A1/MOD13A1_005_2014_05_09');

print(image.projection().getInfo())

visParams = {min: 0.15, max: 0.7}

map9 = folium.Map(location=[38.627, -90.1994], zoom_start=10)
map9.add_ee_layer(image, visParams, 'original')
map9

{'type': 'Projection', 'crs': 'SR-ORG:6974', 'transform': [463.3127165279165, 0, -20015109.353988, 0, -463.3127165274999, 10007554.676994]}


In [41]:
visParams = {min: 0.15, max: 0.7}
reprojected = image.reproject('EPSG:4326')

map10 = folium.Map(location=[38.627, -90.1994], zoom_start=10)
map10.add_ee_layer(image, visParams, 'original')
print(reprojected.projection().getInfo())
map10

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [42]:
image1 = ee.Image('SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL/s01_20150502T082736Z')
image2 = ee.Image('SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL/s01_20150305T081019Z')

image1Orig = image1.resample('bicubic')
image2Orig = image2.resample('bicubic')

image1RedBand = image1Orig.select('R')
image2RedBand = image2Orig.select('R')

print(image1.getInfo())

{'type': 'Image', 'bands': [{'id': 'B', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5376, 11520], 'crs': 'PROJCS["UTM Zone 37, Northern Hemisphere", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      TOWGS84[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Transverse_Mercator"], \n  PARAMETER["central_meridian", 39.0], \n  PARAMETER["latitude_of_origin", 0.0], \n  PARAMETER["scale_factor", 0.9996], \n  PARAMETER["false_easting", 500000.0], \n  PARAMETER["false_northing", 0.0], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]', 'crs_transform': [2, 0, 323332, 0, -2, 77020.8]}, {'id': 'G', 'data_type': {'type': '

In [43]:
displacement = image2RedBand.displacement(referenceImage = image1RedBand, maxOffset = 50.0, patchWidth = 100.0)

registered = image2Orig.displace(displacement)

visParams = {'bands': ['R', 'G', 'B'], max: 4000}
map11 = geemap.Map(center = [0.6166, 37.4579], zoom = 10)
map11.addLayer(image1Orig, visParams, 'References')
map11.addLayer(image2Orig, visParams, 'Before Registration')
map11.addLayer(registered, visParams, 'After Registration')
map11

Map(center=[0.6166, 37.4579], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [44]:
alsoRegistered = image2Orig.register(referenceImage = image1Orig, maxOffset = 50.0, patchWidth = 100.0)

visParams ={'bands': ['R', 'G', 'B'], max: 4000}
map12 = geemap.Map(center = [0.6166, 37.4579], zoom = 10)
map12.addLayer(alsoRegistered, visParams, 'Also Registered')
map12

Map(center=[0.6166, 37.4579], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [45]:
geometry = ee.Geometry.Polygon(
    [[[-76.1397, 43.0399],
      [-76.1399, 43.0314],
      [-76.1283, 43.0314],
      [-76.1284, 43.0397]]]
)

NAIP = ee.ImageCollection('USDA/NAIP/DOQQ')

naipImage = NAIP.filterBounds(geometry).filterDate('2011-01-01', '2011-12-30').mosaic()
map13 = geemap.Map(center = [43.036, -76.136], zoom = 10)
map13.addLayer(geometry, {}, 'ROI')
map13.addLayer(naipImage, {}, 'original NAIP image')
map13

Map(center=[43.036, -76.136], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [46]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [47]:
imageGRN = naipImage.select(['G', 'R', 'N'])

maxValue = imageGRN.reduce(ee.Reducer.max())

imageMAX = naipImage.addBands(maxValue.select(['max']), ['max'])

C3 = imageMAX.expression(
    'atan(B/max)', {
        'B': imageMAX.select('B'),
        'max': imageMAX.select('max')
    }
)


In [48]:
shadowmask = C3.select('B').gte(0.85)
C3shadow = C3.updateMask(shadowmask)

NDWI = imageMAX.expression(
    '(G-N)/(G+N)', {
        'G': imageMAX.select('G'),
        'N': imageMAX.select('N')
    }
)

NDWImask = NDWI.select('G').lte(0.6)
C3shadow = C3shadow.updateMask(NDWImask)

map13 = geemap.Map(center = [43.036, -76.136], zoom = 10)
map13.addLayer(C3shadow)
map13

Map(center=[43.036, -76.136], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [49]:
naipImage = naipImage.select(['R', 'G', 'B']).divide(255)

naipImagehsv = naipImage.rgbToHsv()

NSVDI = naipImagehsv.expression(
    '(S-V)/(S+V)', {
        'S': naipImagehsv.select('saturation'),
        'V': naipImagehsv.select('value')
    }
)

In [50]:
NSVDImask = NSVDI.select('saturation').gte(-0.2)
NSVDIshadow = NSVDI.updateMask(NSVDImask)

ndwi_viz = {'min': 0.5, 'max': 1, 'palette': ['FF0000']}

# Define a map centered on San Francisco Bay.
map_nsvdi = folium.Map(location=[43.036, -76.136], zoom_start=10)

# Add the image layer to the map and display it.
map_nsvdi.add_ee_layer(NSVDIshadow, ndwi_viz, 'NDWI')
display(map_nsvdi)

In [51]:
cloudy_scene = ee.Image('LANDSAT/LT05/C01/T1_TOA/LT05_015030_20100531')
map = geemap.Map(center = [43.046, -76.147], zoom = 10)
map.centerObject(cloudy_scene)

visParams = {'bands': ['B4', 'B3', 'B2'], max: 0.4}

map.addLayer(cloudy_scene, visParams, 'TOA')
map

Map(center=[43.046, -76.147], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [52]:
scored = ee.Algorithms.Landsat.simpleCloudScore(cloudy_scene)

mask50 = scored.select(['cloud']).lte(50)
mask30 = scored.select(['cloud']).lte(30)

masked50 = cloudy_scene.updateMask(mask50)

In [53]:
visParams = {'bands': ['B4', 'B3', 'B2'], max: 0.4}
map.addLayer(masked50, visParams, 'Masked 50')

masked30 = cloudy_scene.updateMask(mask30)
map.addLayer(masked30, visParams, 'Masked 30')

map

Map(center=[43.046, -76.147], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…